In [2]:
from ase.io import read
from hubbard.data import AtomicData
from hubbard.gutz import GhostGutzwiller
from hubbard.utils.tools import setup_seed
from hubbard.utils.tools import get_semicircle_e_list
import numpy as np
from hubbard.data import _keys

setup_seed(123)
U = 0.5
J = 0.0 * U # 0.25 * U
Up = U - 2*J
Jp = J

alpha = 1.
V = 1.0
ep = -1.0
e_list = get_semicircle_e_list(nmesh=1000)

eks = np.zeros((len(e_list), 2, 2))+0j
eks += np.array([[[0.,  V],
                 [V,   ep]]],dtype=np.complex128)
eks[:,1,1] += alpha * e_list

phy_onsite = {
    "C": np.array([
        [[-U/2, 0.],
        [0., 0.]]
    ]),
}

intparams = {"C":[{"U":U,"Up":Up,"J":J, "Jp":Jp}]}

gga = GhostGutzwiller(
    atomic_number=np.array([6]),
    nocc=3,
    basis={"C":[1,1]},
    idx_intorb={"C":[0]},
    naux=3,
    intparams=intparams,
    nspin=2,
    kBT=0.0002,
    mutol=1e-6,
    solver="NQS",
    mixer_options={"method": "Linear", "a": 0.3},
    iscomplex=False,
    solver_options={
        "mfepmax":5,
        "nnepmax":5,
        "d_emb": 4,
        "nblocks":3,
        "hidden_channels":8,
        "out_channels":8,
        "ffn_hidden":[8],
        "heads":4,
        "Ptol": 1e-4, # this value should be smaller than at least 2e-4 if expecting 1e-3 convergence
        "Etol": 6e-4
        },
)

atomicdata = AtomicData.from_ase(
    read("../../hubbard/test/C_cube.vasp"),
    r_max=3.1
    )

# atomicdata["kpoint"] = torch.tensor(kmesh_sampling([10,10,10], True)).to(torch.get_default_dtype())
# block_to_feature(atomicdata, gga.kinetic.idp_phy, block)
atomicdata = AtomicData.to_AtomicDataDict(atomicdata)
atomicdata[_keys.HAMILTONIAN_KEY] = eks
atomicdata[_keys.PHY_ONSITE_KEY] = phy_onsite

# gga.load("./gGA/test/nqs/bethe1o/stateU5P-1P1e-4E1e-3.npz")
gga.run(atomicdata, 500, 1e-3)

/nessa/users/zhanghao/softwares/miniconda3/envs/hubbard/lib/python3.10/site-packages/quantax/sites/sites.py:37: UserWarning: A second 'sites' is defined.
  warn("A second 'sites' is defined.")
/nessa/users/zhanghao/softwares/miniconda3/envs/hubbard/lib/python3.10/site-packages/hubbard/solver/graph_net.py:191: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  layers = [ReshapeSite(dtype), EmbedLayer(d_emb, dtype), *blocks, eqx.nn.Lambda(lambda x: jnp.swapaxes(x, -1, -2)), scale]


DM_kin:  [0.00992037 0.00992037 0.50047442 0.50047442 0.98965505 0.98965505
 0.99994985 0.99994985]
--- Einf: -0.5187, Ehf: -3.0811


Neural Pfaffian training:   0%|          | 0/5 [00:11<?, ?it/s]


RuntimeError: Require a `Lattice`, but got a general `Sites`